In [1]:
import sys
import os
import json
import utils
sys.path.append('c:\\Users\\yurou\\Desktop\\Master thesis\\Non-parametric-Learning-of-DAG-by-RKHS\\Final')
from RKHS import RKHS_DAGMA_extractj
import sys
from notears import NotearsMLP, notears_nonlinear, NotearsSobolev, notears_linear
import torch
import numpy as np
import json
import argparse

In [2]:
torch.set_default_dtype(torch.double)

torch.backends.cudnn.benchmark = False

torch.use_deterministic_algorithms(True)

In [3]:
def squared_loss(output, target):
    n = target.shape[0]
    loss = 0.5 / n * torch.sum((output - target) ** 2)
    return loss


def NOTEARS_MLP(X, B_true, function_type, d, seed):
    results = {}
    model = NotearsMLP(dims=[d, 10, 1], bias=True)
    W_est, output = notears_nonlinear(model, X, lambda1=2e-2)
    acc = utils.count_accuracy(B_true, W_est != 0)
    diff = np.linalg.norm(W_est - abs(B_true))
    X_torch = torch.from_numpy(X)
    x_est = model.forward(X_torch)
    mse = squared_loss(x_est, X_torch)
    h_val = model.h_func().detach().cpu().numpy()
    filename = f'NOTEARS_MLP_function_type_{function_type}_d{d}_seed{seed}'
    results = {
    'SHD': acc['shd'],
    'TPR': acc['tpr'],
    'F1': acc['f1'],
    'diff': diff,
    'mse': mse.item(),
    'h_val': h_val.item(),
    'W_est': W_est.tolist(),
    'B_true': B_true.tolist(),
    'X': X.tolist()}

    with open(filename, 'w') as file:
        json.dump(results, file, indent=4) 

In [7]:
import os

# Current directory where the script is running
current_directory = os.getcwd()

# Relative path to the file
relative_path = 'T6'

# Full path to the file
full_path = os.path.join(current_directory, relative_path)
full_path

'c:\\Users\\yurou\\Desktop\\Master thesis\\Non-parametric-Learning-of-DAG-by-RKHS\\Final\\experiments\\T6'

In [9]:
import json
import numpy as np
from concurrent.futures import ProcessPoolExecutor

def process_NOTEARS_MLP(d, seed):
    function_type = 'gp'
    filename = f'RKHS_function_type_{function_type}_d{d}_seed{seed}'
    file_path = os.path.join(full_path, filename)
    with open(file_path, 'r') as file:
        data = json.load(file)
        B_true = np.array(data['B_true'])
        X = np.array(data['X'])
    
    # Call the NOTEARS_MLP function (assumed to be defined elsewhere)
    NOTEARS_MLP(X, B_true, function_type, d, seed)


In [11]:
process_NOTEARS_MLP(10, 0)

In [14]:
import multiprocessing
d = 10
seeds_range = range(10)
with multiprocessing.Pool() as pool:
        # Map worker function to available seeds, pass `d` as the fixed argument
    pool.starmap(process_NOTEARS_MLP, [(d, seed) for seed in seeds_range])
